In [1]:
import boto3

service_name = 's3'
endpoint_url = 'https://kr.object.ncloudstorage.com'
region_name = 'kr-standard'
#네이버 클라우드 플랫폼 object storage
access_key = ''
secret_key = ''
prefix = 'usa_news_crawling/'

s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                    aws_secret_access_key=secret_key)
bucket_name = 'bucket1chaegpt'

# list all in the bucket
max_keys = 300
response = s3.list_objects(Bucket=bucket_name, MaxKeys=max_keys)

In [20]:
prefix = 'usa_news_crawling/'  # 원하는 폴더 경로

# Prefix를 사용하여 폴더 안에 있는 모든 파일 리스트 가져오기 (페이징)
result_list = []  # 모든 객체를 저장할 리스트
paginator = s3.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

# 제외할 접두사와 접미사의 길이
prefix_length = len("usa_news_crawling/REUTERS_")
suffix_length = len("_2023-08-31.xlsx")

dataframes = []

for page in page_iterator:
    if 'Contents' in page:
        contents_list = page['Contents']
        for content in contents_list:
            key = content['Key']
            # 접두사와 접미사를 제외하고 중간 부분만 추출
            extracted_text = key[prefix_length:-suffix_length]
            result_list.append(extracted_text)

item_list = []
for item in result_list:
    item_list.append(item)




In [6]:
import pandas as pd
df = pd.read_csv('C:/Users/01061.DESKTOP-UR6KUEV/KT_visang/해외_산업분류(추가제거2).csv').iloc[:,1:3]

In [26]:
import pandas as pd
read_file = df
comp_list = read_file.Company

#로이터(Reuters)에 검색하기 위해 기업 공식 표기명 수정
def modify_strings(data_list):
    modified_list = []

    for item in data_list:
        if item.endswith(", Inc."):
            modified_item = item[:-6]  # Remove ", Inc."
        elif item.endswith(" Inc"):
            modified_item = item[:-4]   # Remove " Inc"
        elif item.endswith(" Inc."):
            modified_item = item[:-5]   # Remove " Inc."
        elif item.endswith("corporation"):
            modified_item = item[:-11]  # Remove "corporation"
        elif item.endswith("Corporation"):
            modified_item = item[:-11]  # Remove "Corporation" and add "corp"
            modified_item += "corp"
        elif item.endswith("company"):
            modified_item = item[:-7]   # Remove "company"
        elif item.endswith("Company"):
            modified_item = item[:-7]   # Remove "Company"
        elif item.endswith("Co."):
            modified_item = item[:-3]   # Remove "Co."
        else:
            modified_item = item

        modified_list.append(modified_item)

    return modified_list

# 문자열 수정 및 새로운 리스트 반환
modified_result = modify_strings(comp_list)

def replace_slash_with_space(strings_list):
    modified_strings = []

    for string in strings_list:
        modified_string = string.replace('/', ' ')
        modified_strings.append(modified_string)

    return modified_strings

modified_strings = replace_slash_with_space(modified_result)


In [27]:
df['company2'] = modified_strings

In [28]:
df

,Ticker,Company,company2
0,AA,Alcoa Corporation,Alcoa corp
1,ACNT,Ascent Industries Co.,Ascent Industries
2,ALB,Albemarle Corporation,Albemarle corp
3,ALTO,"Alto Ingredients, Inc.",Alto Ingredients
4,AMR,"Alpha Metallurgical Resources, Inc.",Alpha Metallurgical Resources
...,...,...,...
4873,OBDC,Owl Rock Capital Corporation,Owl Rock Capital corp
4874,HEI.A,HEICO Corporation,HEICO corp
4875,MOG.A,Moog Inc.,Moog
4876,UHAL.B,U-Haul Holding Company,U-Haul Holding


In [43]:
df.to_csv('해외리스트_최종.csv', encoding='utf-8-sig')